In [14]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import tictactoe
importlib.reload(tictactoe);
from tictactoe import TicTacToe


In [15]:
#
# Hyper parameters
# 
alpha = 0.2
gamma = 0.9
epsilon = 0.3

In [16]:
Q = {}
games = 0

In [25]:
#
# TRAINING
#
log_interval = 10000
lastQlen = len(Q)

for _ in range(100000):
    games += 1
    env = TicTacToe()
    done = False
    history = []

    while not done:       
        state = env.state

        if random.uniform(0, 1) < epsilon:
            action = random.choice([a for a in range(9) if env.is_valid(a)])
        else:
            action = max([a for a in range(9) if env.is_valid(a)], key=lambda x: Q.get((state, x), 0))
        
        env.move(action)

        if env.is_won():
            Q[(state, action)] = 1
            next_max = 1
            done = True
        elif env.is_full():
            Q[(state, action)] = 0
            next_max = 0
            done = True
        else:
            oldQ = Q.get((state, action), None)
            next_state = env.state
            next_max = -max([Q.get((next_state, a), 0) for a in range(9) if env.is_valid(a)])
            targetQ = -0.1 + gamma * next_max
            if oldQ is None:
                Q[(state, action)] = targetQ
            else:
                Q[(state, action)] = oldQ + alpha * (targetQ - oldQ)

    if games % log_interval == 0:
        lenQ = len(Q)
        print(f'{games}: Q {lenQ} (+{lenQ-lastQlen}) zeros: {sum(1 for q in Q.values() if q == 0)}')
        lastQlen = lenQ

610000: Q 15499 (+5) zeros: 80
620000: Q 15503 (+4) zeros: 80
630000: Q 15509 (+6) zeros: 80
640000: Q 15514 (+5) zeros: 80
650000: Q 15525 (+11) zeros: 80
660000: Q 15531 (+6) zeros: 80
670000: Q 15534 (+3) zeros: 80
680000: Q 15546 (+12) zeros: 80
690000: Q 15554 (+8) zeros: 80
700000: Q 15559 (+5) zeros: 80


In [32]:
#
#validation
#
wins = 0
draws = 0
losses = 0
for _ in range(10000):
    env = TicTacToe()
    qplayer = random.choice([env.player, env.opponent])
    done = False
    while not done:
        state = env.state
        if qplayer == env.player:
            action = max([a for a in range(9) if env.is_valid(a)], key = lambda x: Q.get((state, x), 0))
        else:
            action = random.choice([a for a in range(9) if env.is_valid(a)])
        env.move(action)
        if env.is_won():
            if qplayer == env.opponent:
                wins += 1
            else:
                losses += 1
            done = True
        elif env.is_full():
            draws += 1
            done = True

print(f'Wins: {wins} Losses: {losses} Draws: {draws}')

Wins: 9384 Losses: 0 Draws: 616


In [158]:
env = TicTacToe()

In [165]:
env.move(7)
env.render()

|O X|
|XXO|
|OX |


In [166]:
#q = model(env.state)
#action = max([a for a in range(9) if env.is_valid(a)], key=lambda x: q[x].item())
state = env.state
for i in range(9):
    if env.is_valid(i):
        print(f'{i}: {Q.get((state, i), 0)}')
action = max([a for a in range(9) if env.is_valid(a)], key=lambda x: Q.get((state,x), 0))
env.move(action)
print(action)
env.render()

1: -0.1
8: -0.9999999999999999
1
|OOX|
|XXO|
|OX |


In [90]:
sum(1 for q in Q.values() if q == 0)

3321

In [134]:
l = [a for a in range(9) if state[a] == ' ']
max([a for a in l], key=lambda x: Q.get((state,action), 0))
l

[1, 2, 3, 6, 7, 8]

In [129]:
x = { 0: -1, 1: -0.5, 2: -1, 3: -0.75}
max([i for i in range(len(x))], key = lambda k: x.get(k, 0))

1